In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Sentence-BERT

In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import json
import numpy as np

In [ ]:
#read file
def prompts_direct_data(path):
  with open(path, 'r', encoding='utf-8') as f:
    prompts_data = json.load(f)
    #captions = [item['captions'] for item in prompts_data]
    return prompts_data

In [ ]:
#compute similarity
def compute_similarity(source_captions, captions):
  source_embeddings = model.encode(source_captions)
  embeddings = model.encode(captions)
  similarities = [model.similarity(source_embeddings[i], embeddings[i]) for i in range(len(source_embeddings))]
  average_similarity = np.mean(similarities)
  var_similarity = np.var(similarities)
  print(average_similarity, var_similarity)

In [ ]:
source_captions = prompts_direct_data('/content/all_test_original.json')
#baseline_captions = prompts_direct_data('/content/all__Flan-T5-XL_baseline.json')
#game_sound_designer_captions = prompts_direct_data('/content/new_all__Flan-T5-XL_game_sound_designer.json')
#sound_designer_captions = prompts_direct_data('/content/all__Flan-T5-XL_sound_designer.json')
#sound_engineer_captions = prompts_direct_data('/content/new_all_Flan-T5-XL_sound_engineer.json')
sound_mixer_captions = prompts_direct_data('/content/empty.json')
#interactive_captions = prompts_direct_data('/content/all__Flan-T5-XL_interactive.json')

In [ ]:
# all
#compute_similarity(source_captions,baseline_captions)
#compute_similarity(source_captions,game_sound_designer_captions)
#compute_similarity(source_captions,sound_designer_captions)
#compute_similarity(source_captions,sound_engineer_captions)
compute_similarity(source_captions,sound_mixer_captions)
#compute_similarity(source_captions,interactive_captions)

0.6102165 0.079813175


# SacreBLEU

In [ ]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
import sacrebleu

In [ ]:
# compute BLEU scores
def calculate_bleu(source_captions, captions):
    # SacreBLEU expects a list of references and each reference is a list of sentences
    return sacrebleu.sentence_bleu(source_captions, [captions]).score

similarity_scores_1 = [calculate_bleu(source_captions[i], baseline_captions[i]) for i in range(len(source_captions))]
average_1 = np.mean(similarity_scores_1)
print(average_1)

similarity_scores_2 = [calculate_bleu(source_captions[i], game_sound_designer_captions[i]) for i in range(len(source_captions))]
average_2 = np.mean(similarity_scores_2)
print(average_2)

similarity_scores_3 = [calculate_bleu(source_captions[i], sound_designer_captions[i]) for i in range(len(source_captions))]
average_3 = np.mean(similarity_scores_3)
print(average_3)

similarity_scores_4 = [calculate_bleu(source_captions[i], sound_engineer_captions[i]) for i in range(len(source_captions))]
average_4 = np.mean(similarity_scores_4)
print(average_4)

similarity_scores_5 = [calculate_bleu(source_captions[i], sound_mixer_captions[i]) for i in range(len(source_captions))]
average_5 = np.mean(similarity_scores_5)
print(average_5)

similarity_scores_6 = [calculate_bleu(source_captions[i], interactive_captions[i]) for i in range(len(source_captions))]
average_6 = np.mean(similarity_scores_6)
print(average_6)
# print BLEU scores
#for i, score in enumerate(similarity_scores):
    #print(f"Sentence 1: {sentence1[i]}")
    #print(f"Sentence 2: {sentence2[i]}")
    #print(f"BLEU Score: {score}")
    #print("-" * 30)


22.66367031308425
22.66367031308425
15.235229055768702
15.235229055768702
22.66367031308425
37.25931701717311


In [ ]:
def calculate_bleu(source_captions, captions):
    # SacreBLEU expects a list of references and each reference is a list of sentences
    return sacrebleu.sentence_bleu(source_captions, [captions]).score

similarity_scores_1 = [calculate_bleu(source_captions[i], sound_mixer_captions[i]) for i in range(len(source_captions))]
average_1 = np.mean(similarity_scores_1)
print(average_1)

23.97597092362101


# self-information: Shannon Entropy

In [ ]:
import nltk
from collections import Counter

nltk.download('punkt')

# tokenize
#tokens = nltk.word_tokenize(text)

# compute frequency
#freq = Counter(tokens)
#total_tokens = len(tokens)

# compute Shannon Entropy
def shannon_entropy(tokens):
    freq = Counter(tokens)
    probs = [freq[token] / len(tokens) for token in freq]
    return -sum(p * np.log2(p) for p in probs)

entropys = []
for caption in baseline_captions:
  tokens = tokens = nltk.word_tokenize(caption)
  entropy = shannon_entropy(tokens)
  entropys.append(entropy)
mean_1 = np.mean(entropys)

print(f"Shannon Entropy: {mean_1}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Shannon Entropy: 2.8783711392889786
